In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
data = pd.read_csv('../input/black-friday-sales-prediction/train_oSwQCTC (1)/train.csv')
data

In [3]:
data.shape

In [4]:
data.info()

# Basic Stats

In [5]:
data.describe().T

# Checking for missing data

In [6]:
data.isnull().sum()/data.shape[0]*100

**There are 31% of product category 2 missing and           69% of product category3**

# Unique Values

In [7]:
data.nunique()

In [8]:
data.Gender.value_counts()

# Exploratory Data Analysis

In [9]:
plt.figure(figsize = (12,8))
plt.title("Purchase Distribution")
sns.distplot(data['Purchase'],color = 'r')

This Disribution kinda follows a normal disribution

# Check for Outliers

In [10]:
plt.figure(figsize = (10,6))
sns.boxplot(x = data['Purchase'],palette = "Set3")
plt.show()

This is Positive Skew

In [11]:
plt.figure(figsize = (12,8))
sns.boxplot(x = 'Age' , y = "Purchase" , data = data , hue = "Gender",palette = "Set3" )
plt.show()

In [12]:
data['Purchase'].skew()

# Analysis By Gender

In [13]:
sns.countplot(data['Gender'])
plt.show()

In [14]:
data.groupby("Gender")["Purchase"].mean()

In [15]:
data['Gender'].value_counts(normalize = True)*100

We can see that the number of males is 3x the number of females

# Marital Status

In [16]:
data['Marital_Status'].value_counts()

In [17]:
sns.countplot(data['Marital_Status'])
plt.show()

**More single people are purchasing**

In [18]:
data.groupby('Marital_Status')["Purchase"].mean()

**They both spend maybe the same amount**

# Occupation

In [19]:
plt.figure(figsize=(12,8))
sns.countplot(data['Occupation'])
plt.show()

In [20]:
jobs = data.groupby("Occupation")["Purchase"].mean()
plt.figure(figsize = (12 , 9))
jobs.plot(kind = 'bar')

**Unfortunatley we don't know the corresponds jobs for each number of those**

# City

In [21]:
sns.countplot(data['City_Category'])

**The most number of purchases came form city B**

In [22]:
plt.figure(figsize = (12,8))
sns.countplot(data['City_Category'],hue = data['Gender'])
plt.show()

# Age

In [23]:
sns.countplot(data['Age'])

In [24]:
data.groupby('Age')['Purchase'].sum().plot(kind = 'bar')

In [25]:
data.groupby('Age')['Purchase'].mean().plot(kind = 'bar')

In [26]:
plt.figure(figsize = (12,8))
sns.countplot(data['Age'],hue = data['Gender'])
plt.show()

# Product Category 1

In [27]:
plt.figure(figsize=(12,6))
sns.countplot(data['Product_Category_1'])
plt.show()

In [28]:
data.groupby('Product_Category_1')['Purchase'].sum().plot(kind='bar',figsize=(18,5))
plt.title("Product 1 Purchases ")
plt.show()

# Product Category 2

In [29]:
plt.figure(figsize=(12,6))
sns.countplot(data['Product_Category_2'])
plt.show()

In [30]:
data.groupby('Product_Category_2')['Purchase'].sum().plot(kind='bar',figsize=(12,8))
plt.title("product 2 purchases")
plt.show()

# Product Category 3

In [31]:
plt.figure(figsize=(12,6))
sns.countplot(data['Product_Category_3'])
plt.show()

In [32]:
data.groupby('Product_Category_3')['Purchase'].sum().plot(kind='bar',figsize=(14,8))
plt.title("The purchases of product 3")
plt.show()

In [33]:
plt.figure(figsize=(12,6))
sns.heatmap(data.corr(),annot = True,cmap = 'BuPu')

# Data Preprocessing

In [34]:
df = data.copy()

In [35]:
df.isnull().sum()

In [36]:
df['Age'].value_counts()

In [37]:
df.drop(['User_ID','Product_ID'],axis = 1, inplace = True)
df

In [38]:
df.Stay_In_Current_City_Years.replace({'0':0,
                                         '1':1,
                                         '2':2,
                                         '3':3,
                                         '4+':4},inplace = True)
df.Gender.replace({'M':1,'F':0},inplace = True)
df.City_Category.replace({'A':1,'B':2,'C':3},inplace = True)
df.Age.replace({'0-17' :17,
                  '18-25':20,
                  '26-35':30,
                  '36-45':40,
                  '46-50':47,
                  '51-55':52,
                  '55+' : 56},
                 inplace = True)

In [39]:
df

In [40]:
df.info()

**Now the data doesn't contain any objects **

In [41]:
df.drop('Product_Category_3',axis = 1,inplace = True)#We dropped this column becaue 2/3 of the data is missing


In [42]:
df['Product_Category_2'] =df['Product_Category_2'].fillna(0)

In [43]:
df

# Splitting the data to Train and Test

In [44]:
X = df.drop("Purchase",axis = True)
y = df["Purchase"]

In [45]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)

In [2]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [47]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report,plot_confusion_matrix
from xgboost import XGBClassifier

In [48]:
score = {}
pipe = make_pipeline(StandardScaler(),KNeighborsClassifier())
pipe.fit(X_train,y_train)
score['KNN'] = pipe.score(X_test,y_test)

In [ ]:
pipe = make_pipeline(StandardScaler(),LogisticRegression())
pipe.fit(X_train,y_train)
score['logistic_Regression'] = pipe.score(X_test,y_test)